In [2]:
# stl-SSNP-12B2-cut-plates-along-Y-axis_dxf.py
# 2021-12-25
#
# pickle 是 Python 的內建模組
# read 349x196x3_XYZ.dat file
#
# Autodesk DWG Trueview
# zoom extents --> can see all drawing.
# The most important step to take with a drawing you receive from a colleague, consultant, or client
# is to Zoom to Extents.
#
import os
path = os.getcwd()
print('當前工作目錄 ==>', path)
#
from shapely.geometry  import LineString
from operator import itemgetter
#
import pprint
import pickle
import ezdxf
from ezdxf import units
#
path_data    = './data/'
path_graphic = './graphic/'
#
"""
# AC1027 ---> AutoCAD R2013
# line weight ---> 0.13mm = 13
# minimum line weight 13 
# maximum line width 200
#
# color  0 --> Black  ex:cut
# color  1 --> red
# color  3 --> green  ex:carve
# color  5 --> Dark blue
# color  6 --> Magenta
"""
#
## ================================================================
#
with open(path_data + '349x196x3_XYZ.dat', 'rb') as in_file :
    dem_rows = pickle.load(in_file)
# end with    
#
# up layer verteices
A_up = [38.0, 90.0, 0.0] # 0
B_up = [-1.0, 90.0, 0.0] # 1
C_up = [-1.0, 20.4, 0.0] # 2
D_up = [38.0, 20.4, 0.0] # 3
#
# down layer verteices
a_dn = [38.0, 90.0,-2.0] # 4
b_dn = [-1.0, 90.0,-2.0] # 5
c_dn = [-1.0, 20.4,-2.0] # 6
d_dn = [38.0, 20.4,-2.0] # 7
#
left_open_cut_xy  = [(10.25, -2) , (10.25, -1.2), ( 9.75, -1.2), ( 9.75, -2)]
left_dn_xy  = [(-1.0,-2.0)]
#
right_dn_xy = [(38.0,-2.0)]
right_open_cut_xy = [(28.25, -2) , (28.25, -1.2), (27.75, -1.2), (27.75, -2)]
#
xz_rows = []
for i in range(0,349,1) :
    XZ_tuple_row = [(XYZ[0], round(XYZ[2],3)) for XYZ in dem_rows[i]]
    Boundary = left_open_cut_xy + left_dn_xy + XZ_tuple_row + right_dn_xy + right_open_cut_xy + [(10.25, -2)]
    xz_rows.append(Boundary)
# end for
#
print('Laser cut boundary has', len(xz_rows),    'rows')
print('         each rows has', len(xz_rows[0]), 'columns')
print ('\n')
#
y_max_list = []
for row in xz_rows :
    y_max_list.append(max(row,key=itemgetter(1))[1])
# end for
# print('total', len(rows))
# print(y_max_list)
#
ezdoc = ezdxf.new('R2013', setup=True)
# Set centimeter as document/modelspace units
ezdoc.units = units.CM
# which is a shortcut (including validation) for
ezdoc.header['$INSUNITS'] = units.CM  # 5
#
ezmsp = ezdoc.modelspace()
ezdoc.layers.new(name='SSNP', 
                 dxfattribs={'linetype': 'CONTINUOUS', 
                            'lineweight': 150,
                            'color': 1})
#
i = 0
y_sum = 0.0
data_rows = xz_rows[i:i+15]
#data_rows = rows[i:]  # for last plate
for row in data_rows :
    if len(str(i)) == 1 :
        i_str = "00" + str(i)
    elif len(str(i)) == 2 :
        i_str = "0" + str(i)
    else :
        i_str = str(i)
    # end if
    #
    # text height default is 0.25 
    if i > 0 :
        row = [(xy[0], xy[1]+y_sum) for xy in row]
    # end if
    #
    if i == 0 :
        text_pos = (15.0, -1.3+y_sum)
        text_str = 'width 39 cm   height 2 cm'
        ezmsp.add_text(text_str, 
                   dxfattribs={ 
                   'style': 'LiberationSerif',
                   'height': 0.5,
                   'color': 3}
                  ).set_pos(text_pos, align='MIDDLE_RIGHT')
    # end if
    text_str = i_str
    text_pos = (4.0, -1.3+y_sum)
    ezmsp.add_lwpolyline(row, dxfattribs={'layer': 'SSNP'})
    ezmsp.add_text(text_str, 
                   dxfattribs={ 
                   'style': 'LiberationSerif',
                   'height': 0.5,
                   'color': 3}
                  ).set_pos(text_pos, align='MIDDLE_RIGHT')
    #
    y_sum = y_sum + 2 + y_max_list[i] + 0.5
    # i = i + 1  # for last plate
    if y_sum + 2 + y_max_list[i+1] + 0.8 > 30.0 :
        break
    else :
        i = i + 1
    #end if
# end for
#
file_id = i_str + '.dxf'
ezdoc.saveas(path_graphic + "Laser-cut/row-" + file_id)
#
print(y_sum)
print("Done")


當前工作目錄 ==> /home/jovyan
Laser cut boundary has 349 rows
         each rows has 207 columns


27.718
Done
